# ch03 객체 생성 팩토리 만들기

- 팩토리
  - 객체 지향 개발 용어
  - 다른 객체를 생성하기 위한 클래스를 의미
  - 이 클래스는 파라미터를 받고, 그 파라미터에 따라 객체를 반환하는 메소드를 가지고 있음
  
### 이 장에서 다룰 내용

- 간단한 팩토리를 만드는 방법
- 팩토리 메소드가 무엇이고, 언제 사용하는지, 다양한 웹 자원과 연결하는 도구를 만들기 위한 구현 방법
- 추상 팩토리가 무엇이고, 언제 사용하는지, 그리고 팩토리 메소드 패턴과 다른점은 무엇인지?

### 객체를 직접 인스턴스화하기보다 팩토리를 사용해야 할 이유는?

- 팩토리는 객체 생성과 클래스 구현을 구별해서 의존성을 낮춤
- 생성된 객체를 사용하는 클래스는 정확히 어떤 클래스에서 객체를 생성했는지 알 필요 없음. 생성된 클래스의 인터페이스인, 어떤 속성으로 어떤 생성 클래스의 메소드를 호출할 수 있는지만 알면 됨
- 새로운 클래스가 인터페이스에 부합하기만 한다면, 클라이언트 코드를 수정하지 않고 팩토리에서 새로운 클래스를 추가할 수 있음
- Factory 클래스는 기존 객체를 재사용할 수 있지만, 직접 인스턴스화하기는 언제나 새로운 객체를 생성한다.

In [7]:
%%writefile src/simple_factory.py

# coding: utf-8

class SimpleFactory(object):
    # 이 데코레이터는 메소드가 클래스 인스턴스없이 실행할 수 있게 한다.
    # ex: SimpleFactory.build_connection
    @staticmethod 
    def build_connection(protocol):
        if protocol == 'http':
            return HTTPConnection()
        elif protocol == 'ftp':
            return FTPConnection()
        else:
            raise RuntimeError('Unknown protocol')
            

if __name__ == '__main__':
    protocol = raw_input('Which Protocol to use? (http or ftp): ')
    protocol = SimpleFactory.build_connection(protocol)
    protocol.connect()
    print protocol.get_response()

Overwriting src/simple_factory.py


In [8]:
%run src/simple_factory.py

Which Protocol to use? (http or ftp): http


NameError: global name 'HTTPConnection' is not defined

## 팩토리 메소드

- 객체 생성을 책임지는 factory\_method를 포함한 추상 클래스 Creator를 가지고 있음
- some\_operation 메소드는 생성된 객체를 가지고 작업을 함
- ConcreateCreator 클래스는 factory\_method를 재정의해서 생성된 객체를 실행시간에 변경함
- some\_operation 메소드는 Product 인터페이스를 구현하고 모든 메소드에 이를 제공하기만 한다면 어떤 객체가 생성되었는지에는 관여하지 않음
- 핵심: 객체 생성을 위한 인터페이스를 정의하는 것이지만, 인터페이스를 구현하는 클래스에게 어떤 클래스를 인스턴스화 할 것인지 결정하도록 함
- 이 인터페이스는 Creator와 ConcreateCreator 클래스의 factory\_method이고, 생성할 Product의 서브클래스를 결정함
- 팩토리 메소드는 상속에 기반을 함. 객체 생성은 Factory 메소드를 구현하는 서브 클래스에 위임함

### 팩토리 메소드 패턴의 장점

- 코드를 좀 더 일반적으로 만들어 실제 클래스(ConcreteProduct)에 묶이지 않고 인터페이스(Product)의 의존도를 낮춤. 인터페이스를 구현으로부터 분리
- 객체를 생성하는 코드와 사용하는 코드를 분리해서 관리가 쉽도록 함
- 새로운 클래스를 추가하려면 elif 구문 하나만 추가하면 됨

### 팩토리 메소드 구현

- HTTP와 FTP 프로토콜을 사용해서 웹 자원에 접근하는 도구를 만들어 본다.
- 추상 클래스 Creator를 만들고 Connector 라고 이름 붙임
- 이 클래스는 원격 자원(HTTP 혹은 FTP)에 연결을 하고, 응답을 읽고, 파싱하는 역할을 함
- 이 추상 클래스는 어떤 포트와 프로토콜을 사용해야 할지 모름
- 왜냐하면 HTTP의 표준 포트는 80이고 FTP는 21이기 때문
- 또한 HTTP의 프로토콜을 http 이거나 https가 되고 FTP의 프로토콜을 ftp가 됨
- 따라서 어떤 포트를 사용해야 할지는 실행 시간에 자식 클래스에서 결정

In [24]:

# coding: utf-8
import abc
import urllib2
from BeautifulSoup import BeautifulSoup
from urlparse import urlparse
from urlparse import urlunparse


class Connector(object):
    """원격 서버에 연결하기 위한 추상 클래스"""
    __metaclass__ = abc.ABCMeta # Declare class as abstract class
    
    def __init__(self, is_secure):
        self.is_secure = is_secure
        self.port = self.port_factory_method()
        self.protocol = self.protocol_factory_method()
        
    @abc.abstractmethod
    def parse(self):
        """웹 콘텐츠를 파싱한다.
        이 메소든느 실행 시간에 재정의 해야만 한다."""
        pass
    
    def read(self, host, path):
        """모든 서브클래스에 대한 일반 메소드로, 웹 콘텐츠를 읽는다."""
        url = urlunparse((self.protocol, host+':'+str(self.port), path, 
                         '', '', ''))
#         url = self.protocol + '://' + host + ':' + str(self.port) + path
        print('Connecting to ', url)
        return urllib2.urlopen(url, timeout=2).read()
    
    @abc.abstractmethod
    def protocol_factory_method(self):
        """서브클래스에서 반드시 재정의해야 하는 팩토리 메소드"""
        pass
    
    @abc.abstractmethod
    def port_factory_method(self):
        """서브클래스에서 반드시 재정의해야 하는 또 다른 팩토리 메소드"""
        return FTPPort()

In [25]:
class HTTPConnector(Connector):
    """HTTP 커넥터를 생성하고 모든 속성을 실행 시간에 설정하는 실제 생성자"""
    def protocol_factory_method(self):
        if self.is_secure:
            return 'https'
        return 'http'
    
    def port_factory_method(self):
        """HTTPPort와 HTTPSecurePort는 실제 객체로, 팩토리 메소드에서 생성한 것이다."""
        if self.is_secure:
            return HTTPSecurePort()
        return HTTPPort()
    
    def parse(self, content):
        """웹 콘텐츠 파싱"""
        soup = BeautifulSoup(content)
        links = soup.table.findAll('a')
        filenames = [link['href'] for link in links]
        return '\n'.join(filenames)
    

class FTPConnector(Connector):
    """FTP 커넥터를 생성하고 모든 속성을 실행 시간에 설정하는 실제 생성자"""
    def protocol_factory_method(self):
        return 'ftp'
    
    def port_factory_method(self):
        return FTPPort()
    
    def parse(self, content):
        lines = content.split('\n')
        filenames = []
        for line in lines:
            # 일반적으로 FTP 포맷은 열을 8개 가지고 있음. 이것을 나누어 주자.
            splitted_line = line.split(None, 8)
            if len(splitted_line) == 9:
                filenames.append(splitted_line[-1])
        return '\n'.join(filenames)

In [26]:
class Port(object):
    __metaclass__ = abc.ABCMeta
    """추상 생성물. 이 중 하나의 서브클래스는 팩토리 메소드에서 생성된다."""
    
    @abc.abstractmethod
    def __str__(self):
        pass
    

class HTTPPort(Port):
    """http 포트를 나타내는 실제 생성물"""
    def __str__(self):
        return '80'
    
    
class HTTPSecurePort(Port):
    """https 포트를 나타내는 실제 생성물"""
    def __str__(self):
        return '443'
    
    
class FTPPort(Port):
    """ftp 포트를 나타내는 실제 생성물"""
    def __str__(self):
        return '21'

In [28]:
if __name__== '__main__':
    domain = 'ftp.freebsd.org'
    path = '/pub/FreeBSD/'
    
    protocol = input('Connecting to {}. Which Protocol to use? (0-http, 1-ftp): '.format(domain))
    
    if protocol == 0:
        is_secure = bool(input('Use secure connection? (1-yes, 0-no): '))
        connector = HTTPConnector(is_secure)
    else:
        is_secure = False
        connector = FTPConnector(is_secure)
        
    try:
        content = connector.read(domain, path)
    except urllib2.URLError as e:
        print('Can not access resource with this method')
    else:
        print(connector.parse(content))

Connecting to ftp.freebsd.org. Which Protocol to use? (0-http, 1-ftp): 1
('Connecting to ', 'ftp://ftp.freebsd.org:21/pub/FreeBSD/')
README.TXT
TIMESTAMP
development
dir.sizes
doc
ports
releases



## 추상 팩토리

- 팩토리 메소드의 목적: 인스턴스 생성 과정을 서브클래스로 이동시키는 것
- 추상 팩토리의 목적: 특정 클래스에 의존하지 않고 연관 객체의 가족을 만드는 것
- AbstractFactory 인터페이스에서 상속받은 모든 팩토리는 AbstractProduct와 AnotherAbstractProduct 인터페이스를 생성하기 위한 메소드를 가지고 있음
- 추상 팩토리에서 생성한 객체는 동일한 인터페이스를 가져야 하지만 생성한 실제 객체는 팩토리마다 달라야 함을 전제로 함
- 따라서 서로 다른 동작성을 원한다면, 실행 시간에 팩토리를 변경하고 변경된 객체를 얻을 수 있음
- 추상 팩토리는 서로 협업하는 객체 가족을 만들 때 주로 사용함
- 추상 팩토리를 사용하면 클라이언트로부터 객체 생성을 분리해서, 클라이언트가 인터페이스를 통해서만 접근할 수 있도록 만들어 추후 조작이 용이하다는 장점이 있음
- 만약 생성한 객체들이 서로 협업해야 한다면, AbstractFactory 클래스가 동시에 하나의 객체만 사용해서 작업을 쉽게 만든다. 
- 한편으로 기존 팩토리에 새로운 것을 추가하기는 쉽지 않은데, 이는 AbstractFactory 인터페이스가 사용하는 생성물 집합이 고정적이기 때문
- 따라서 새로운 생성물을 추가하기 위해서는 팩토리 인터페이스를 확장해야 하고, AbstractFactory 클래스와 그 서브클래스를 모두 수정해야 함

### 추상 팩토리 패턴의 장점

- 생성물 가족을 치환하는 과정을 쉽게 만듦
- 생성물 가족의 요소 간 호환성을 높임
- 실제 클래스와 클라이언트를 분리함

### 추상 팩토리 구현

In [29]:
import abc
import urllib2
from BeautifulSoup import BeautifulStoneSoup


class AbstractFactory(object):
    """추상 팩토리 인터페이스는 서브클래스에서 3 가지 메소드를 제공함
    create_protocol, create_port, create_parser."""
    __metaclass__ = abc.ABCMeta
    
    def __init__(self, is_secure):
        """is_secure가 True라면, 팩토리는 보안 연결을 사용함"""
        self.is_secure = is_secure
        
    @abc.abstractmethod
    def create_protocol(self):
        pass
    
    @abc.abstractmethod
    def create_port(self):
        pass
    
    @abc.abstractmethod
    def create_parser(self):
        pass

In [30]:
class HTTPFactory(AbstractFactory):
    """HTTP 연결을 위한 실제 팩토리"""
    def create_protocol(self):
        if self.is_secure:
            return 'https'
        return 'http'
    
    def create_port(self):
        if self.is_secure:
            return HTTPSecurePort()
        return HTTPPort()
    
    def create_parser(self):
        return HTTPParser()

In [31]:
class FTPFactory(AbstractFactory):
    """FTP 연결을 위한 실제 팩토리"""
    def create_protocol(self):
        return 'ftp'
    
    def create_port(self):
        return FTPPort()
    
    def create_parser(self):
        return FTPParser()

In [36]:
class Port(object):
    __metaclass_ = abc.ABCMeta
    """연결할 포트를 나타내는 추상 생성물. 팩토리 메소드에서 이것의 서브클래스를 생성한다."""
    
    @abc.abstractmethod
    def __str__(self):
        pass
    

class HTTPPort(Port):
    """http 포트를 나타내는 실제 생성물"""
    def __str__(self):
        return '80'


class HTTPSecurePort(Port):
    """https 포트를 나타내는 실제 생성물"""
    def __str__(self):
        return '443'


class FTPPort(Port):
    """ftp 포트를 나타내는 실제 생성물"""
    def __str__(self):
        return '21'

In [37]:
class Parser(object):
    """웹 콘텐츠를 파싱하는 파서를 나타내는 추상 생성물.
    이것의 서브클래스가 팩토리 메소드에서 생성된다."""
    __metaclass__ = abc.ABCMeta
    
    @abc.abstractmethod
    def __call__(self, content):
        pass
    

class HTTPParser(Parser):
    def __call__(self, content):
        soup = BeautifulStoneSoup(content)
        links = soup.table.findAll('a')
        filenames = [link.text for link in links]
        return '\n'.join(filenames)
    
    
class FTPParser(Parser):
    def __call__(self, content):
        lines = content.split('\n')
        filenames = []
        for line in lines:
            splitted_line = line.split(None, 8)
            if len(splitted_line) == 9:
                filenames.append(splitted_line[-1])
        return '\n'.join(filenames)

In [39]:
class Connector(object):
    """클라이언트"""
    def __init__(self, factory):
        """팩토리는 AbstractFactory 인스턴스로, 팩토리 클래스에 따라 
        커넥터의 모든 속성을 생성한다."""
        self.protocol = factory.create_protocol()
        self.port = factory.create_port()
        self.parse = factory.create_parser()
        
    def read(self, host, path):
        url = self.protocol + '://' + host + ':' + str(self.port) + path
        print('Connecting to ', url)
        return urllib2.urlopen(url, timeout=2).read()
    
    @abc.abstractmethod
    def parse(self):
        pass    

In [43]:
if __name__ == '__main__':
    domain = 'ftp.freebsd.org'
    path = '/pub/FreeBSD/'
    
    protocol = input('Connecting to {}. Which Protocol to use? (0-http, 1-ftp): '.format(domain))
    
    if protocol == 0:
        is_secure = bool(input('Use secure connection? (1-yes, 0-no): '))
        factory = HTTPFactory(is_secure)
    elif protocol == 1:
        is_secure = False
        factory = FTPFactory(is_secure)
    else:
        print('Sorry, wrong answer')
        
    connector = Connector(factory)
    try:
        content = connector.read(domain, path)
    except urllib2.URLError as e:
        print('Can not access resource with this method')
    else:
        print(connector.parse(content))

Connecting to ftp.freebsd.org. Which Protocol to use? (0-http, 1-ftp): 0
Use secure connection? (1-yes, 0-no): 0
('Connecting to ', 'http://ftp.freebsd.org:80/pub/FreeBSD/')
File Name
&nbsp;↓&nbsp;
File Size
&nbsp;↓&nbsp;
Date
&nbsp;↓&nbsp;
Parent directory/
development/
doc/
ports/
releases/
snapshots/
README.TXT
TIMESTAMP
dir.sizes


## 추상 팩토리와 팩토리 메소드

- 팩토리 메소드 패턴 
  - 클라이언트와 클라이언트가 사용하는 생성물을 구분지어야 할 때 사용함
  - 클라이언트로 하여금 인스턴스의 생성과 설정의 책임을 갖지 않게 할 때 팩토리 메소드를 사용함
- 추상 팩토리 패턴
  - 클라이언트를 생성물 클래스에서 반드시 분리해야 할 때 사용함
  - 또한 추상 팩토리 패턴은 어떤 클래스를 객체의 독립적 가족 생성과 함께 사용해야 할지 강제할 수 있음

## 요약

- 객체지향 개발 용어중 팩토리란 다른 클래스를 생성하는 클래스를 가리킴
- 팩토리 메소드는 객체 생성을 위한 인터페이스를 정의하지만, 어떤 클래스를 인스턴스화할지는 인터페이스를 구현하는 클래스에서 결정함
- 팩토리 메소드는 코드를 실제 클래스가 아닌 인터페이스에 묶어 좀 더 보편적으로 사용할 수 있게 만듦
- 추상 팩토리는 실제 클래스를 명시하지 않은 상태로 관련있는 가족이나 의존된 객체를 생성하는 인터페이스를 제공함
- 생성물 가족의 대체품을 단순화하고 가족을 이루는 생성물의 호환성을 보장함